<a href="https://colab.research.google.com/github/Art-phys/Lesson_HF_LR/blob/main/Lesson_HF_RL_Unit8_part2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Блок 8, Часть 2: Углубленное обучение с подкреплением. Использование Sample Factory для воспроизведения Doom из пикселей

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit9/thumbnail2.png" alt="Thumbnail" width="70%"/>

В этом блокноте мы узнаем, как обучить глубокую нейронную сеть собирать объекты в 3D-среде на основе игры Doom, видео результирующей политики показано ниже. Мы разрабатываем эту политику, используя [Sample Factory](https://www.samplefactory.dev/), асинхронная реализация алгоритма PPO.

Пожалуйста, обратите внимание на следующие моменты:

*   [Sample Factory](https://www.samplefactory.dev/) это продвинутый фреймворк RL и **работает только в Linux и Mac** (не в Windows).

*  Фреймворк лучше всего работает на ** машине с графическим процессором и большим количеством процессорных ядер **, где он может достигать скорости 100 тыс. взаимодействий в секунду. Ресурсы, доступные в стандартном ноутбуке Colab **, ограничивают производительность этой библиотеки**. Таким образом, скорость в этом параметре ** не отражает реальную производительность **.
* Контрольные показатели для Sample Factory доступны в ряде настроек, ознакомьтесь с [примерами](https://github.com/alex-petrenko/sample-factory/tree/master/sf_examples) если вы хотите узнать больше.


In [ ]:
from IPython.display import HTML

HTML('''<video width="640" height="480" controls>
  <source src="https://huggingface.co/edbeeching/doom_health_gathering_supreme_3333/resolve/main/replay.mp4" 
  type="video/mp4">Your browser does not support the video tag.</video>'''
)

Чтобы подтвердить это практическое руководство для [процесса сертификации](https://huggingface.co/deep-rl-course/en/unit0/introduction#certification-process), вам нужно обучить модель:

- `doom_health_gathering_supreme` получив результат >= 5.

Чтобы найти свой результат, перейдите к [leaderboard](https://huggingface.co/spaces/huggingface-projects/Deep-Reinforcement-Learning-Leaderboard) и найдите свою модель, **the result = mean_reward - std of reward**

Прежде чем приступить к обучению нашего агента, давайте ** изучим библиотеку и среды, которые мы собираемся использовать**.

## Фабрика образцов

[Sample Factory](https://www.samplefactory.dev/) является одной из **самых быстрых библиотек RL, ориентированных на очень эффективные синхронные и асинхронные реализации градиентов политики (PPO)**.

Sample Factory (Фабрика образцов) тщательно **протестирована, используется многими исследователями и практиками** и активно поддерживается. Известно, что наша реализация **обеспечивает производительность SOTA в различных областях при минимизации времени обучения RL-эксперименту и требований к оборудованию**.

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit9/samplefactoryenvs.png" alt="Sample factory" width="70%"/>


### Ключевые функции

- Высокооптимизированный алгоритм [архитектура](https://www.samplefactory.dev/06-architecture/overview/) для максимальной производительности обучения
- [Синхронный и асинхронный](https://www.samplefactory.dev/07-advanced-topics/sync-async/) режимы обучения
- [Последовательный (однопроцессорный) режим](https://www.samplefactory.dev/07-advanced-topics/serial-mode/) для удобства отладки
- Оптимальная производительность как в средах на базе CPU, так и [в средах с ускорением GPU](https://www.samplefactory.dev/09-environment-integrations/isaacgym/)
- Обучение с одним и несколькими агентами, самостоятельная игра, поддержка [обучения нескольким политикам](https://www.samplefactory.dev/07-advanced-topics/multi-policy-training/) одновременно на одном или нескольких графических процессорах
- Обучение на уровне населения ([PBT](https://www.samplefactory.dev/07-advanced-topics/pbt/))
- Дискретные, непрерывные, гибридные пространства действий
- Векторные, основанные на изображениях, словари пространства наблюдений
- Автоматически создает архитектуру модели путем анализа спецификации пространства действий/наблюдения. Поддерживает [архитектуры пользовательских моделей](https://www.samplefactory.dev/03-customization/custom-models/)
- Предназначен для импорта в другие проекты, [пользовательские среды](https://www.samplefactory.dev/03-customization/custom-environments/)
- Детализация [WandB and Tensorboard summaries](https://www.samplefactory.dev/05-monitoring/metrics-reference/), [пользовательские показатели](https://www.samplefactory.dev/05-monitoring/custom-metrics/)
- [HuggingFace 🤗 интеграция](https://www.samplefactory.dev/10-huggingface/huggingface/) (загрузка обученных моделей и метрик на Hub)
- [Множество](https://www.samplefactory.dev/09-environment-integrations/mujoco/) [примеров](https://www.samplefactory.dev/09-environment-integrations/atari/) [окружающих сред](https://www.samplefactory.dev/09-environment-integrations/vizdoom/) [для интеграции](https://www.samplefactory.dev/09-environment-integrations/dmlab/) с настроенными параметрами и обученными моделями

Все вышеуказанные политики доступны на 🤗 hub. Выполните поиск по тегу[sample-factory](https://huggingface.co/models?library=sample-factory&sort=downloads)

### Как работает sample-factory

Sample-factory - это одна из **наиболее высокооптимизированных реализаций RL, доступных сообществу**.

Это работает путем **порождения нескольких процессов, которые запускают работников развертывания, работников вывода и работника-учащегося**.

*Рабочие* **взаимодействуют через общую память, что снижает стоимость связи между процессами**.

*Работники развертывания* взаимодействуют с окружающей средой и отправляют наблюдения *работникам вывода*.

Работники *вывода* запрашивают фиксированную версию политики и **отправляют действия обратно работнику развертывания**.

После *k* шагов работы по развертыванию отправляют траекторию опыта работнику-учащемуся**, которую он использует для обновления сети политик агента**.

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit9/samplefactory.png" alt="Sample factory" width="70%"/>

### Модели актер-критик из Sample-factory

Модели актера и критика в Sample Factory состоят из трех компонентов:

- ** Кодировщик ** - Обрабатывает входные наблюдения (изображения, векторы) и преобразует их в вектор. Это та часть модели, которую вы, скорее всего, захотите настроить.
- ** Ядро ** - Интегрирует векторы из одного или нескольких кодеров, может дополнительно включать одно- или много- слойный LSTM/GRU в агента на основе памяти.
- **Декодер** - Использует дополнительные слои к выходным данным ядра модели перед вычислением выходных данных политики и значения.

Библиотека была разработана для автоматической поддержки любых пространств наблюдения и действий. Пользователи могут легко добавлять свои пользовательские модели. Вы можете узнать больше в [документации](https://www.samplefactory.dev/03-customization/custom-models/#actor-critic-models-in-sample-factory).

## ViZDoom

[ViZDoom](https://vizdoom.cs.put.edu.pl/) is an **open-source python interface for the Doom Engine**. 

The library was created in 2016 by Marek Wydmuch, Michal Kempka  at the Institute of Computing Science, Poznan University of Technology, Poland. 

The library enables the **training of agents directly from the screen pixels in a number of scenarios**, including team deathmatch, shown in the video below. Because the ViZDoom environment is based on a game the was created in the 90s, it can be run on modern hardware at accelerated speeds, **allowing us to learn complex AI behaviors fairly quickly**.

The library includes feature such as:

- Multi-platform (Linux, macOS, Windows),
- API for Python and C++,
- [OpenAI Gym](https://www.gymlibrary.dev/) environment wrappers
- Easy-to-create custom scenarios (visual editors, scripting language, and examples available),
- Async and sync single-player and multiplayer modes,
- Lightweight (few MBs) and fast (up to 7000 fps in sync mode, single-threaded),
- Customizable resolution and rendering parameters,
- Access to the depth buffer (3D vision),
- Automatic labeling of game objects visible in the frame,
- Access to the audio buffer
- Access to the list of actors/objects and map geometry,
- Off-screen rendering and episode recording,
- Time scaling in async mode.